In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import GridSearchCV
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import precision_recall_curve, auc, roc_curve

import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM, GRU
from tensorflow.python.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow import keras
from sklearn.metrics import f1_score
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding
from tensorflow.keras.models import Model

from tensorflow.keras.layers import *

pd.set_option('display.max_columns', None)
plt.rcParams['figure.dpi'] = 120
plt.rcParams['savefig.dpi'] = 120

In [4]:
pd.set_option('display.max_columns', None)

## Load the data (ready for modeling)

In [5]:
#df_model = pd.read_csv('model.csv', nrows=5)
df_model = pd.read_csv('model.csv')

In [6]:
#df_model[df_model.isnull().any(axis=1)]
df_model = df_model.dropna()
df_model

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_common,word_total,word_share,freq_q1+q2,freq_q1-q2,question1_tokenize,question2_tokenize,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,tdidf_0,tdidf_1,tdidf_2,tdidf_3,tdidf_4,tdidf_5,tdidf_6,tdidf_7,tdidf_8,tdidf_9,tdidf_10,tdidf_11,tdidf_12,tdidf_13,tdidf_14,tdidf_15,tdidf_16,tdidf_17,tdidf_18,tdidf_19,tdidf_20,tdidf_21,tdidf_22,tdidf_23,tdidf_24,tdidf_25,tdidf_26,tdidf_27,tdidf_28,tdidf_29,tdidf_30,tdidf_31,tdidf_32,tdidf_33,tdidf_34,tdidf_35,tdidf_36,tdidf_37,tdidf_38,tdidf_39,tdidf_40,tdidf_41,tdidf_42,tdidf_43,tdidf_44,tdidf_45,tdidf_46,tdidf_47,tdidf_48,tdidf_49,tdidf_50,tdidf_51,tdidf_52,tdidf_53,tdidf_54,tdidf_55,tdidf_56,tdidf_57,tdidf_58,tdidf_59,tdidf_60,tdidf_61,tdidf_62,tdidf_63,tdidf_64,tdidf_65,tdidf_66,tdidf_67,tdidf_68,tdidf_69,tdidf_70,tdidf_71,tdidf_72,tdidf_73,tdidf_74,tdidf_75,tdidf_76,tdidf_77,tdidf_78,tdidf_79,tdidf_80,tdidf_81,tdidf_82,tdidf_83,tdidf_84,tdidf_85,tdidf_86,tdidf_87,tdidf_88,tdidf_89,tdidf_90,tdidf_91,tdidf_92,tdidf_93,tdidf_94,tdidf_95,tdidf_96,tdidf_97,tdidf_98,tdidf_99,tdidf_100,tdidf_101,tdidf_102,tdidf_103,tdidf_104,tdidf_105,tdidf_106,tdidf_107,tdidf_108,tdidf_109,tdidf_110,tdidf_111,tdidf_112,tdidf_113,tdidf_114,tdidf_115,tdidf_116,tdidf_117,tdidf_118,tdidf_119,tdidf_120,tdidf_121,tdidf_122,tdidf_123,tdidf_124,tdidf_125,tdidf_126,tdidf_127,tdidf_128,tdidf_129,tdidf_130,tdidf_131,tdidf_132,tdidf_133,tdidf_134,tdidf_135,tdidf_136,tdidf_137,tdidf_138,tdidf_139,tdidf_140,tdidf_141,tdidf_142,tdidf_143,tdidf_144,tdidf_145,tdidf_146,tdidf_147,tdidf_148,tdidf_149,tdidf_150,tdidf_151,tdidf_152,tdidf_153,tdidf_154,tdidf_155,tdidf_156,tdidf_157,tdidf_158,tdidf_159,tdidf_160,tdidf_161,tdidf_162,tdidf_163,tdidf_164,tdidf_165,tdidf_166,tdidf_167,tdidf_168,tdidf_169,tdidf_170,tdidf_171,tdidf_172,tdidf_173,tdidf_174,tdidf_175,tdidf_176,tdidf_177,tdidf_178,tdidf_179,tdidf_180,tdidf_181,tdidf_182,tdidf_183,tdidf_184,tdidf_185,tdidf_186,tdidf_187,tdidf_188,tdidf_189,tdidf_190,tdidf_191
0,0.0,1.0,2.0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0.0,1.0,1.0,66.0,57.0,14.0,12.0,10.0,23.0,0.434783,2.0,0.0,"['step', 'step', 'guid', 'invest', 'share', 'm...","['step', 'step', 'guid', 'invest', 'share', 'm...",0.833319,0.714276,0.999980,0.999980,0.833326,0.714281,0.0,1.0,2.0,13.0,100.0,93.0,93.0,98.0,0.965517,18.807824,41.964858,33.081644,4.244245,43.027614,-10.013357,-10.744914,-11.815834,5.588284,1.115507,-30.170084,-14.351320,41.116792,-7.119523,3.995967,33.354323,-17.730944,5.780393,1.179608,31.968379,-13.365348,-21.448225,0.158826,26.865440,3.917031,6.164278,-48.380915,-51.427958,-5.240506,-29.813020,47.343491,-13.405041,23.369998,-20.407769,52.206585,-40.527661,3.913583,-20.964618,-9.969450,24.479733,-21.353528,-40.896076,4.871263,12.776786,-38.494769,-7.711000,-36.135508,-41.597447,65.922466,-11.392577,-15.133901,7.812485,-20.850897,1.058450,10.323407,-24.293006,-38.674469,10.414390,-2.161339,-26.870887,5.204462,-37.068663,20.469225,-23.657622,-10.948505,3.628947,-5.457772,-30.500613,1.938033,-42.096031,-14.017004,-22.719999,-16.798715,15.333135,8.758236,7.055754,-14.903748,16.649700,27.768037,-40.461602,54.995131,10.239546,14.740792,2.994080,2.752803,-30.065014,20.576211,-21.568772,-24.228445,14.157072,-16.261099,25.543985,25.480805,39.165051,3.834179,9.871825,18.710793,37.375936,45.405470,5.427006,33.751087,-3.253411,-4.959901,-13.442779,3.905698,4.777377,-22.148543,-8.397876,33.991805,-15.717423,0.769366,20.965538,-8.149176,-6.982945,-0.220294,34.898826,-13.520903,-20.837738,-1.247399,19.857070,5.311532,5.790612,-45.432595,-45.181854,0.781877,-22.808248,44.329990,-18.003015,16.850946,-31.704211,44.328688,-31.347574,-3.478976,-23.225434,-6.520577,20.683242,-13.918028,-34.093079,12.777202,7.638465,-38.681467,-

## Split data, prepare X_train, y_train, X_test, y_test

In [7]:
X = df_model.drop(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate', 'question1_tokenize', 'question2_tokenize'], axis=1)
y = df_model['is_duplicate']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 23)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((282998, 218), (121286, 218), (282998,), (121286,))

## LogisticRegression

In [6]:
#clf_log = LogisticRegression(random_state=23, C=100)
clf_log = SGDClassifier(alpha=1, penalty='l2', loss='log', random_state=23)
clf_log.fit(X_train, y_train)
clf_sig = CalibratedClassifierCV(clf_log, method="sigmoid")
clf_sig.fit(X_train, y_train)
y_pred_log = clf_sig.predict_proba(X_test)
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, np.where(y_pred_log[:,0] > 0.5, 0, 1))))
print('F1 score: {}'. format(f1_score(y_test, np.where(y_pred_log[:,0] > 0.5, 0, 1))))
print('Model log loss: {}'. format(log_loss(y_test, y_pred_log, eps=1e-15)))
cm_log = confusion_matrix(y_test, np.where(y_pred_log[:,0] > 0.5, 0, 1))
print('\nConfusion matrix\n', cm_log)

Model accuracy score: 0.7685
F1 score: 0.6584006619293293
Model log loss: 0.45188499967508733

Confusion matrix
 [[66157 10168]
 [17906 27055]]


#### grid search

In [ ]:
log_parameter_candidates = [
    {'C': [1e-1, 10, 100]}
]

gridsearch_log = GridSearchCV(estimator=clf_log, param_grid=log_parameter_candidates, cv=2, scoring='neg_log_loss', verbose=10)
gridsearch_log.fit(X_train, y_train)   

# print('Best solver:',gridsearch_log.best_estimator_.solver) 
# print('Best penalty:',gridsearch_log.best_estimator_.penalty)
print('Best C:',gridsearch_log.best_estimator_.C)
print('Score:', gridsearch_log.score(X_test, y_test))
y_pred_log_best = gridsearch_log.predict(X_test)
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_log_best)))
print('Model log loss: {}'. format(log_loss(y_test, y_pred_log_best, eps=1e-15)))
cm_log_best = confusion_matrix(y_test, y_pred_log_best)
print('\nConfusion matrix\n', cm_log_best)

## Random Forest 

In [ ]:
clf_rf = RandomForestClassifier(max_depth=100, min_samples_split=5, n_estimators=100, random_state=23)
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_test)
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_rf)))
print('F1 score: {}'. format(f1_score(y_test, y_pred_rf)))
y_pred_rf_proba = clf_rf.predict_proba(X_test)
print('Model log loss: {}'. format(log_loss(y_test, y_pred_rf_proba, eps=1e-15)))
cm_rf = confusion_matrix(y_test, y_pred_rf)
print('\nConfusion matrix\n', cm_rf)

#### grid search

In [ ]:
rf_parameter_candidates = [
    {'n_estimators': [100, 300, 500],
     'max_depth': [10, 100, None],
     'bootstrap': [True, False],
     'max_features': ['auto', 'sqrt']
    }
]

gridsearch_rf = GridSearchCV(estimator=clf_rf, param_grid=rf_parameter_candidates, cv=2, scoring='logloss', verbose=10)
gridsearch_rf.fit(X_train, y_train)   

#print('Best number of trees:',gridsearch_rf.best_estimator_.n_estimators) 
print('Best max depth:',gridsearch_rf.best_estimator_.max_depth)
print('Best bootstrap:',gridsearch_rf.best_estimator_.bootstrap)
print('Best max_features:',gridsearch_rf.best_estimator_.max_features)
# print('Best min_samples_leaf:',gridsearch_rf.best_estimator_.min_samples_leaf)
# print('Best min_samples_split:',gridsearch_rf.best_estimator_.min_samples_split)
print('Score:', gridsearch_rf.score(X_test, y_test))
y_pred_rf_best = gridsearch_rf.predict(X_test)
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_rf_best)))
print('Model log loss: {}'. format(log_loss(y_test, y_pred_rf_best, eps=1e-15)))
cm_rf_best = confusion_matrix(y_test, y_pred_rf_best)
print('\nConfusion matrix\n', cm_rf_best)

## XGBooster

In [ ]:
clf_xgb = xgb.XGBClassifier(eval_metric='logloss', n_estimators=100, gamma=0, max_depth=7, learning_rate=0.2, random_state=23)
clf_xgb.fit(X_train,y_train)
y_pred_xgb = clf_xgb.predict(X_test)
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_xgb)))
print('F1 score: {}'. format(f1_score(y_test, y_pred_xgb)))
y_pred_xgb_proba = clf_xgb.predict_proba(X_test)
print('Model log loss: {}'. format(log_loss(y_test, y_pred_xgb_proba, eps=1e-15)))
cm_xgb = confusion_matrix(y_test, y_pred_xgb)
print('\nConfusion matrix\n', cm_xgb)

#### grid search

In [ ]:
xgb_parameter_candidates = [
  {'min_child_weight': [1, 5, 10],
   'subsample': [0.6, 0.8, 1.0],
   'colsample_bytree': [0.6, 0.8, 1.0]}
]

'n_estimators': 90, 'max_depth': 7, 'learning_rate': 0.2, 'gamma': 0

gridsearch_xgb = GridSearchCV(estimator=clf_xgb, param_grid=xgb_parameter_candidates, cv=1, scoring='logloss', verbose=10)
gridsearch_xgb.fit(X_train, y_train)   

print('Best min_child_weight:',gridsearch_xgb.best_estimator_.min_child_weight) 
print('Best gamma:',gridsearch_xgb.best_estimator_.gamma)
print('Best subsample:',gridsearch_xgb.best_estimator_.subsample)
print('Best colsample_bytree:',gridsearch_xgb.best_estimator_.colsample_bytree)
print('Best max_depth:',gridsearch_xgb.best_estimator_.max_depth)
print('Score:', gridsearch_xgb.score(X_test, y_test))
y_pred_xgb_best = gridsearch_xgb.predict(X_test)
print('Model log loss: {}'. format(log_loss(y_test, y_pred_xgb_best, eps=1e-15)))
cm_xgb_best = confusion_matrix(y_test, y_pred_xgb_best)
print('\nConfusion matrix\n', cm_xgb_best)

## LSTM (Model1)

In [8]:
X_train2 = X_train.to_numpy().reshape((X_train.to_numpy().shape[0], 1, X_train.to_numpy().shape[1]))
X_test2 = X_test.to_numpy().reshape((X_test.to_numpy().shape[0], 1, X_test.to_numpy().shape[1]))

In [34]:
DROPOUT = 0.2

model = keras.Sequential()

# model.add(Embedding(input_dim = len(word_index)+1,
#                        output_dim = 200,
#                        weights = [embedding_matrix],
#                        input_length = 218))

model.add(Bidirectional(LSTM(32, return_sequences=True, activation='tanh'), input_shape=(X_train2.shape[1], X_train2.shape[2])))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(LSTM(32, return_sequences=False)))
model.add(Dropout(rate=DROPOUT))

model.add(Dense(units=1))

model.add(Activation('linear'))
#model.add(Activation('LeakyReLU'))

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_16 (Bidirecti  (None, 1, 64)            64256     
 onal)                                                           
                                                                 
 dropout_15 (Dropout)        (None, 1, 64)             0         
                                                                 
 bidirectional_17 (Bidirecti  (None, 1, 64)            24832     
 onal)                                                           
                                                                 
 dropout_16 (Dropout)        (None, 1, 64)             0         
                                                                 
 bidirectional_18 (Bidirecti  (None, 1, 64)            24832     
 onal)                                                           
                                                     

In [35]:
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam'
)

In [ ]:
mc = ModelCheckpoint('best_LSTM_2.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

BATCH_SIZE = 64
#BATCH_SIZE = 32

history = model.fit(
    X_train2, 
    y_train, 
    epochs=300, 
    batch_size=BATCH_SIZE, 
    validation_split=0.1,
    callbacks=[mc]
)

In [32]:
model = load_model('best_LSTM_2.h5')
model.evaluate(X_test2, y_test)
y_pred_LSTM = model.predict(X_test2)
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, np.where(y_pred_LSTM > 0.5, 1, 0))))
print('F1 score: {}'. format(f1_score(y_test, np.where(y_pred_LSTM > 0.5, 1, 0))))
print('Model log loss: {}'. format(log_loss(y_test, y_pred_LSTM, eps=1e-15)))
cm_LSTM = confusion_matrix(y_test, np.where(y_pred_LSTM > 0.5, 1, 0))
print('\nConfusion matrix\n', cm_LSTM)

3791/3791 [==============================] - 8s 2ms/step - loss: 0.4007
Model accuracy score: 0.8051
F1 score: 0.7365735169585952
Model log loss: 0.41050394674899676

Confusion matrix
 [[64591 11734]
 [11908 33053]]


## LSTM (Model2)

In [12]:
embedding_index = {}
with open('glove.6B.200d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [13]:
MAX_NB_WORDS = 200000
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
word_index = tokenizer.word_index
embedding_matrix = np.random.random((len(word_index)+1, 200))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# X_train3 = X_train.to_numpy().reshape((X_train.to_numpy().shape[0], 1, X_train.to_numpy().shape[1]))
# X_test3 = X_test.to_numpy().reshape((X_test.to_numpy().shape[0], 1, X_test.to_numpy().shape[1]))

In [20]:
X_train3_q1 = X_train[['tdidf_0',
'tdidf_1',
'tdidf_2',
'tdidf_3',
'tdidf_4',
'tdidf_5',
'tdidf_6',
'tdidf_7',
'tdidf_8',
'tdidf_9',
'tdidf_10',
'tdidf_11',
'tdidf_12',
'tdidf_13',
'tdidf_14',
'tdidf_15',
'tdidf_16',
'tdidf_17',
'tdidf_18',
'tdidf_19',
'tdidf_20',
'tdidf_21',
'tdidf_22',
'tdidf_23',
'tdidf_24',
'tdidf_25',
'tdidf_26',
'tdidf_27',
'tdidf_28',
'tdidf_29',
'tdidf_30',
'tdidf_31',
'tdidf_32',
'tdidf_33',
'tdidf_34',
'tdidf_35',
'tdidf_36',
'tdidf_37',
'tdidf_38',
'tdidf_39',
'tdidf_40',
'tdidf_41',
'tdidf_42',
'tdidf_43',
'tdidf_44',
'tdidf_45',
'tdidf_46',
'tdidf_47',
'tdidf_48',
'tdidf_49',
'tdidf_50',
'tdidf_51',
'tdidf_52',
'tdidf_53',
'tdidf_54',
'tdidf_55',
'tdidf_56',
'tdidf_57',
'tdidf_58',
'tdidf_59',
'tdidf_60',
'tdidf_61',
'tdidf_62',
'tdidf_63',
'tdidf_64',
'tdidf_65',
'tdidf_66',
'tdidf_67',
'tdidf_68',
'tdidf_69',
'tdidf_70',
'tdidf_71',
'tdidf_72',
'tdidf_73',
'tdidf_74',
'tdidf_75',
'tdidf_76',
'tdidf_77',
'tdidf_78',
'tdidf_79',
'tdidf_80',
'tdidf_81',
'tdidf_82',
'tdidf_83',
'tdidf_84',
'tdidf_85',
'tdidf_86',
'tdidf_87',
'tdidf_88',
'tdidf_89',
'tdidf_90',
'tdidf_91',
'tdidf_92',
'tdidf_93',
'tdidf_94',
'tdidf_95']]

In [21]:
X_train3_q2 = X_train[['tdidf_96',
'tdidf_97',
'tdidf_98',
'tdidf_99',
'tdidf_100',
'tdidf_101',
'tdidf_102',
'tdidf_103',
'tdidf_104',
'tdidf_105',
'tdidf_106',
'tdidf_107',
'tdidf_108',
'tdidf_109',
'tdidf_110',
'tdidf_111',
'tdidf_112',
'tdidf_113',
'tdidf_114',
'tdidf_115',
'tdidf_116',
'tdidf_117',
'tdidf_118',
'tdidf_119',
'tdidf_120',
'tdidf_121',
'tdidf_122',
'tdidf_123',
'tdidf_124',
'tdidf_125',
'tdidf_126',
'tdidf_127',
'tdidf_128',
'tdidf_129',
'tdidf_130',
'tdidf_131',
'tdidf_132',
'tdidf_133',
'tdidf_134',
'tdidf_135',
'tdidf_136',
'tdidf_137',
'tdidf_138',
'tdidf_139',
'tdidf_140',
'tdidf_141',
'tdidf_142',
'tdidf_143',
'tdidf_144',
'tdidf_145',
'tdidf_146',
'tdidf_147',
'tdidf_148',
'tdidf_149',
'tdidf_150',
'tdidf_151',
'tdidf_152',
'tdidf_153',
'tdidf_154',
'tdidf_155',
'tdidf_156',
'tdidf_157',
'tdidf_158',
'tdidf_159',
'tdidf_160',
'tdidf_161',
'tdidf_162',
'tdidf_163',
'tdidf_164',
'tdidf_165',
'tdidf_166',
'tdidf_167',
'tdidf_168',
'tdidf_169',
'tdidf_170',
'tdidf_171',
'tdidf_172',
'tdidf_173',
'tdidf_174',
'tdidf_175',
'tdidf_176',
'tdidf_177',
'tdidf_178',
'tdidf_179',
'tdidf_180',
'tdidf_181',
'tdidf_182',
'tdidf_183',
'tdidf_184',
'tdidf_185',
'tdidf_186',
'tdidf_187',
'tdidf_188',
'tdidf_189',
'tdidf_190',
'tdidf_191']]

In [14]:
word_index = tokenizer.word_index

In [38]:
model_q1 = keras.Sequential()
model_q1.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 200,
                      weights = [embedding_matrix],
                      input_length = 96))
model_q1.add(LSTM(128, activation = 'tanh', return_sequences = True))
model_q1.add(Dropout(0.2))
model_q1.add(LSTM(128, return_sequences = True))
model_q1.add(LSTM(128))
model_q1.add(Dense(60, activation = 'tanh'))
model_q1.add(Dense(2, activation = 'sigmoid'))

model_q2 = keras.Sequential()
model_q2.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 200,
                      weights = [embedding_matrix],
                      input_length = 96))

model_q2.add(LSTM(128, activation = 'tanh', return_sequences = True))
model_q2.add(Dropout(0.2))
model_q2.add(LSTM(128, return_sequences = True))
model_q2.add(LSTM(128))
model_q2.add(Dense(60, activation = 'tanh'))
model_q2.add(Dense(2, activation = 'sigmoid'))

mergedOut = Multiply()([model_q1.output, model_q2.output])

mergedOut = Flatten()(mergedOut)
mergedOut = Dense(100, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.2)(mergedOut)
mergedOut = Dense(50, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.2)(mergedOut)
mergedOut = Dense(2, activation = 'sigmoid')(mergedOut)

In [39]:
new_model = Model([model_q1.input, model_q2.input], mergedOut)
new_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 embedding_6_input (InputLayer)  [(None, 96)]        0           []                               
                                                                                                  
 embedding_7_input (InputLayer)  [(None, 96)]        0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 96, 200)      200         ['embedding_6_input[0][0]']      
                                                                                                  
 embedding_7 (Embedding)        (None, 96, 200)      200         ['embedding_7_input[0][0]']      
                                                                                            

In [ ]:
new_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',
                 metrics = ['accuracy'])
history = new_model.fit([X_train3_q1.to_numpy(),X_train3_q2.to_numpy()],y_train, batch_size = 2000, epochs = 10)

In [ ]:
model = load_model('best_LSTM_2.h5')
model.evaluate(X_test2, y_test)
y_pred_LSTM2 = model.predict(X_test2)
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, np.where(y_pred_LSTM2 > 0.5, 1, 0))))
print('F1 score: {}'. format(f1_score(y_test, np.where(y_pred_LSTM2 > 0.5, 1, 0))))
print('Model log loss: {}'. format(log_loss(y_test, y_pred_LSTM2, eps=1e-15)))
cm_LSTM2 = confusion_matrix(y_test, np.where(y_pred_LSTM2 > 0.5, 1, 0))
print('\nConfusion matrix\n', cm_LSTM2)